# Data Download and Exploration Notebook
## Team 12 Final Project

Step 0: Importing Packages

In [1]:

import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile

c:\Users\ryant\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


This is a link explaining [the wrds package](https://pypi.org/project/wrds/)

In [2]:
# db = wrds.Connection()
# db.list_libraries()

Donwloading CSVs

In [3]:
ipo_age_df = pd.read_csv('inputs/IPO-age(9).csv')
SPACs_df = pd.read_csv('inputs/SPACs2016-2021.csv')
# ipo_age_df
# SPACs_df

Data Exploration

In [4]:
ipo_age_df.describe()
ipo_age_df.info()
ipo_age_df.nunique()
# unnamed columns = notes in the .csv file, we can drop these
ipo_age_df.drop(columns=['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'], inplace=True)
ipo_age_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15447 entries, 0 to 15446
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   offer date         15447 non-null  int64  
 1   IPO name           15447 non-null  object 
 2   ticker             15035 non-null  object 
 3   CUSIP              15317 non-null  object 
 4   Rollup             15447 non-null  object 
 5   VC                 15447 non-null  object 
 6   Dual               15447 non-null  object 
 7   Internet           15447 non-null  object 
 8   Post-issue shares  15447 non-null  object 
 9   CRSP perm          15447 non-null  object 
 10  Founding           15447 non-null  int64  
 11  Unnamed: 11        0 non-null      float64
 12  Unnamed: 12        17 non-null     object 
 13  Unnamed: 13        3 non-null      object 
dtypes: float64(1), int64(2), object(11)
memory usage: 1.7+ MB


,offer date,IPO name,ticker,CUSIP,Rollup,VC,Dual,Internet,Post-issue shares,CRSP perm,Founding
0,19750130,ROYSTER,NaN,78088610,.,0,0,0,.,67898,1901
1,19750609,VARCO,VRC,92212610,.,0,0,0,.,63044,1908
2,19750610,COORS ADOLPH,TAP,21701610,.,0,1,0,35418915,59248,1901
3,19750715,KEYSTONE FOODS CORP,NaN,49348410,.,0,0,0,.,61989,1965
4,19750826,C. F. BRAUN,NaN,10564710,.,0,0,0,.,58579,1909
...,...,...,...,...,...,...,...,...,...,...,...
15442,20231214,INNO HOLDINGS INC.,INHD,4576JP109,.,0,0,0,.,.,2021
15443,20231215,Bayview Acquisition,BAYAU,NaN,.,0,1,0,7732500,.,2023
15444,20231215,ZKH GROUP LTD,ZKH,98877R104,.,0,1,0,5621490964,.,2021
15445,20231219,Linkage Global Inc,LGCB,G5500B102,.,0,0,0,.,.,2022


In [5]:
SPACs_df.describe()
SPACs_df.info()
SPACs_df.nunique() # 6 repeating CUSIPS? 881 total values, 875 unique?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SPAC IPO     1013 non-null   object
 1   Date of IPO  1013 non-null   object
 2   CUSIP        881 non-null    object
dtypes: object(3)
memory usage: 23.9+ KB


SPAC IPO       1013
Date of IPO     423
CUSIP           875
dtype: int64

Downloading the ccm_youngfirms data (courtesy of Prof. Bowen)

In [6]:
ccm_youngfirms = "inputs/ccm_youngfirms_2000_2018.dta/ccm_youngfirms_2000_2018.dta"

if not os.path.exists(ccm_youngfirms):
    zip_path = "inputs/ccm_youngfirms_2000_2018.zip"

    with zipfile.ZipFile(zip_path,'r') as zip_ref:
        zip_ref.extractall("inputs/ccm_youngfirms_2000_2018.dta")
        

In [7]:
data = pd.read_stata(ccm_youngfirms)

ccm_youngfirms is huge and we need to be dealing with less than 950 vars

In [8]:
data.dropna(axis=1,how='all',inplace=True)
# 950 to 589 vars --> still too many

.dropna() with these params drops all the cols with 0 non-NA values.

In [9]:
sample = data.iloc[:,0:20]
# sample.nunique()
isna = sample.isna().sum()/len(sample)
isna

lpermno     0.000000
lpermco     0.000000
gvkey       0.000000
datadate    0.000000
fyear       0.000000
indfmt      0.000000
consol      0.000000
popsrc      0.000000
datafmt     0.000000
tic         0.000000
cusip       0.000000
conm        0.000000
acctchg     0.000000
acctstd     0.000000
acqmeth     0.000000
adrr        0.915863
ajex        0.000000
ajp         0.000000
bspr        0.000000
compst      0.000000
dtype: float64

here we drop all the columns with higher than 90% NA values.

In [10]:
na_percentage = data.isna().mean() * 100

# Filter columns where the percentage of NA values is greater than 90%
columns_with_high_na = na_percentage[na_percentage > 90].index.tolist()

print("Columns with more than 90% NA values:")
print(len(columns_with_high_na))

data.drop(columns=columns_with_high_na, inplace=True)
data

Columns with more than 90% NA values:
165


,lpermno,lpermco,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,...,prcc_f,prch_f,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso
0,85073.0,32010.0,002731,2000-12-31,2000,INDL,C,D,STD,PVN,...,57.50,67.0000,29.0625,1.0,NaN,,,,,
1,85073.0,32010.0,002731,2001-12-31,2001,INDL,C,D,STD,PVN,...,3.55,64.0625,2.0000,1.0,NaN,,,,,
2,85073.0,32010.0,002731,2002-12-31,2002,INDL,C,D,STD,PVN,...,6.49,8.4900,2.5000,1.0,NaN,,,,,
3,85073.0,32010.0,002731,2003-12-31,2003,INDL,C,D,STD,PVN,...,11.64,13.1400,4.8700,1.0,1.0,,,,Y,Y
4,14055.0,54485.0,003169,2013-12-31,2013,INDL,C,D,STD,OMED.2,...,29.52,32.3890,12.0650,1.0,1.0,4,1,0,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27783,17808.0,56369.0,324684,2018-12-31,2018,INDL,C,D,STD,ASLN,...,3.60,10.4390,2.5100,0.2,1.0,5,1,0,Y,Y
27784,17676.0,56329.0,326688,2018-12-31,2018,INDL,C,D,STD,NVT,...,22.46,29.8400,19.7100,1.0,1.0,5,4,1,Y,Y
27785,17884.0,56407.0,327451,2018-12-31,2018,INDL,C,D,STD,GRIN,...,6.24,20.5000,5.3200,1.0,1.0,5,1,0,Y,Y
27786,18313.0,56612.0,328032,2018-12-31,2018,INDL,C,D,STD,NMCI,...,2.83,4.8500,2.5100,1.0,1.0,4,1,0,Y,Y


In [11]:
data.columns.to_list()

['lpermno',
 'lpermco',
 'gvkey',
 'datadate',
 'fyear',
 'indfmt',
 'consol',
 'popsrc',
 'datafmt',
 'tic',
 'cusip',
 'conm',
 'acctchg',
 'acctstd',
 'acqmeth',
 'ajex',
 'ajp',
 'bspr',
 'compst',
 'curcd',
 'curncd',
 'currtr',
 'final',
 'fyr',
 'ismod',
 'ltcm',
 'ogm',
 'pddur',
 'scf',
 'src',
 'stalt',
 'udpl',
 'upd',
 'apdedate',
 'fdate',
 'pdate',
 'acchg',
 'acdo',
 'aco',
 'acodo',
 'acominc',
 'acox',
 'act',
 'aldo',
 'am',
 'ano',
 'ao',
 'aocidergl',
 'aociother',
 'aocipen',
 'aocisecgl',
 'aodo',
 'aol2',
 'aoloch',
 'aox',
 'ap',
 'apalch',
 'aqa',
 'aqc',
 'aqd',
 'aqeps',
 'aqi',
 'aqp',
 'aqpl1',
 'aqs',
 'at',
 'aul3',
 'bkvlps',
 'caps',
 'capx',
 'capxv',
 'ceq',
 'ceql',
 'ceqt',
 'ch',
 'che',
 'chech',
 'ci',
 'cibegni',
 'cicurr',
 'cidergl',
 'cimii',
 'ciother',
 'cipen',
 'cisecgl',
 'citotal',
 'cld2',
 'cld3',
 'cld4',
 'cld5',
 'cogs',
 'cshfd',
 'cshi',
 'csho',
 'cshpri',
 'cshr',
 'cstk',
 'cstkcv',
 'cstke',
 'dc',
 'dclo',
 'dcom',
 'dcpstk'

In [12]:
# variables we need:
relevant_vars = ['adrr','curuscn','scf','src','apdedate','fdate','pdate','acominc',
                 'acox','at','am','ao','aoloch','aox','ap','at','caps','capx','cb',
                 'ch','che','clg','cogs','csho','cshrt','cstk','dd','dlc','dn','do',
                 'dt','ebit','ebtida','epspi','fca','ffo','gdwl','gp','ib','intan',
                 'invt','lt','lct','ni','niadj','np','pi','ppegt','pnrsho','ppent',
                 're','revt','sale','seq','tdc','teq','tstk','txt','wcap','naicsh',
                 'mkvalt','acchg','accrt','amc','ano','arce','cshi','depc','derhedgl']

cleaned_data = data[[col for col in data.columns if col in relevant_vars]]
cleaned_data

,scf,src,apdedate,fdate,pdate,acchg,acominc,acox,am,ano,...,re,revt,sale,seq,teq,tstk,txt,wcap,naicsh,mkvalt
0,NaN,53.0,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,2035.297,5883.549,5883.549,2032.183,NaN,5.976,434.460,NaN,522110.0,16439.8825
1,NaN,8.0,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,1981.166,5529.908,5529.908,1907.511,NaN,76.517,92.330,NaN,522110.0,1011.1323
2,NaN,53.0,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,2212.848,4072.630,4072.630,2139.072,NaN,76.685,49.006,NaN,522110.0,1878.1411
3,NaN,5.0,2003-12-31,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,2337.966,2781.408,2781.408,2325.449,NaN,15.425,128.088,NaN,522110.0,3380.8031
4,7.0,5.0,2013-12-31,2014-04-14,NaT,0.0,0.014,2.527,0.000,0.0,...,-174.412,37.779,37.779,118.122,118.122,0.000,1.944,256.727,325414.0,867.8290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27783,7.0,5.0,2018-12-31,2019-06-26,2019-03-23,0.0,0.000,0.000,0.007,0.0,...,-132.469,0.000,0.000,30.618,30.618,0.000,0.015,21.095,325414.0,NaN
27784,7.0,5.0,2018-12-31,2019-03-01,2019-01-31,0.0,-107.800,12.300,60.900,0.0,...,-24.400,2213.600,2213.600,2687.100,2687.100,0.000,37.900,384.800,3323.0,3980.4510
27785,7.0,5.0,2018-12-31,2019-06-14,NaT,0.0,-4.150,7.595,0.017,0.0,...,-11.190,319.018,319.018,292.503,292.503,0.000,1.389,66.283,483111.0,118.9594
27786,7.0,5.0,2018-12-31,2019-04-08,2019-03-10,0.0,0.000,2.903,33.146,0.0,...,NaN,133.921,133.921,182.473,182.473,0.000,0.000,-22.247,483111.0,97.9265


In [13]:
cleaned_data.nunique()/len(cleaned_data)
cleaned_data.isna().sum()/len(cleaned_data)

scf         0.161581
src         0.107420
apdedate    0.365086
fdate       0.500648
pdate       0.726249
acchg       0.221391
acominc     0.236397
acox        0.224629
am          0.369872
ano         0.720815
ao          0.108212
aoloch      0.164244
aox         0.222650
ap          0.113322
at          0.108212
caps        0.141428
capx        0.168526
ch          0.120052
che         0.108212
cogs        0.112207
cshi        0.111919
csho        0.002951
cstk        0.137361
dd          0.234382
dlc         0.109148
dn          0.236145
do          0.112135
dt          0.232151
ebit        0.124154
epspi       0.114834
fca         0.673096
gdwl        0.174608
gp          0.112207
ib          0.112135
intan       0.141428
invt        0.118756
lct         0.312473
lt          0.109832
ni          0.112135
niadj       0.112135
np          0.111199
pi          0.112135
pnrsho      0.414459
ppegt       0.257161
ppent       0.133943
re          0.138477
revt        0.112135
sale        0

Merging Data

In [14]:
import chardet


try:
    # Trying to read with default UTF-8 encoding first
    naics = pd.read_csv('inputs/2022_Codes.csv')
except UnicodeDecodeError:
    # If UTF-8 fails, try reading with 'ISO-8859-1'
    naics = pd.read_csv('inputs/2022_Codes.csv', encoding='ISO-8859-1')
    
with open('inputs/2022_Codes.csv', 'rb') as file:
    print(chardet.detect(file.read())) 



{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [15]:
naics.head()


,Seq. No.,Code,Industry,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,1.0,11,"Agriculture, Forestry, Fishing and Hunting",NaN,NaN
2,2.0,111,Crop Production,NaN,NaN
3,3.0,1111,Oilseed and Grain Farming,NaN,NaN
4,4.0,11111,Soybean Farming,NaN,NaN


In [ ]:
data.rename(columns={'naicsh': 'Code'}, inplace=True)
data.columns.to_list()

In [26]:
merged_df = pd.concat([data, naics], axis=1)
merged_df.dropna

<bound method DataFrame.dropna of        lpermno  lpermco   gvkey   datadate  fyear indfmt consol popsrc  \
0      85073.0  32010.0  002731 2000-12-31   2000   INDL      C      D   
1      85073.0  32010.0  002731 2001-12-31   2001   INDL      C      D   
2      85073.0  32010.0  002731 2002-12-31   2002   INDL      C      D   
3      85073.0  32010.0  002731 2003-12-31   2003   INDL      C      D   
4      14055.0  54485.0  003169 2013-12-31   2013   INDL      C      D   
...        ...      ...     ...        ...    ...    ...    ...    ...   
27783  17808.0  56369.0  324684 2018-12-31   2018   INDL      C      D   
27784  17676.0  56329.0  326688 2018-12-31   2018   INDL      C      D   
27785  17884.0  56407.0  327451 2018-12-31   2018   INDL      C      D   
27786  18313.0  56612.0  328032 2018-12-31   2018   INDL      C      D   
27787  18144.0  56541.0  328795 2018-12-31   2018   INDL      C      D   

      datafmt     tic  ... au auop auopic ceoso cfoso  Seq. No.   Code  \
0  

In [28]:
merged_df.to_csv('inputs/practice_file.csv', index=False)



In [30]:
lu = pd.read_csv("inputs/practice_file.csv")

lu.isna().sum()

C:\Users\ryant\AppData\Local\Temp\ipykernel_9996\4136096740.py:1: DtypeWarning: Columns (30,425,426,428) have mixed types. Specify dtype option on import or set low_memory=False.
  lu = pd.read_csv("inputs/practice_file.csv")


lpermno           0
lpermco           0
gvkey             0
datadate          0
fyear             0
              ...  
Seq. No.      25663
Code.1        25663
Industry      25663
Unnamed: 3    27788
Unnamed: 4    27787
Length: 429, dtype: int64